# Mosaico 2021

In [2]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

In [3]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Estudio 

In [4]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Funciones

In [5]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [6]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [7]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [8]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [9]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [10]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [34]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [35]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10)) 

In [16]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 2021

In [42]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [43]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [44]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [45]:
coleccion21 = coleccion2021.map(Cloud10)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [46]:
# coleccion21 = coleccion2021.map(Cloud10)
greenest21_v2 = coleccion2021.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest21_v2, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [48]:
coleccion21_ = coleccion2021.map(Cloud30)
agua21 = coleccion21_.qualityMosaic('NDWI')
map_2021_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021_.add_ee_layer(agua21, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [49]:
# coleccion21_ = coleccion2021.map(Cloud30)
agua21_v2 = coleccion2021.qualityMosaic('NDWI')
map_2021_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021_.add_ee_layer(agua21_v2, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [38]:
Map.addLayer(agua21, vizParams, 'agua 2021')
Map.addLayer(coleccion2021, vizParams, 'coleccion 2021')
Map.addLayer(agua21_v2, vizParams, 'agua v2')
Map.addLayer(greenest21, vizParams, 'greenest 2021')
Map

Map(bottom=22040.0, center=[-50.59718623058703, -71.73508418289748], controls=(WidgetControl(options=['positio…

### 2020

In [24]:
start_date = '2019-10-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
coleccion2020 = l8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [53]:
# coleccion20 = coleccion2020.map(Cloud10)
greenest20_v2 = coleccion2020.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020.add_ee_layer(greenest20_v2, vizParams, 'Mapa sin Nubes')
display(map_2020)

In [52]:
coleccion20 = coleccion2020.map(Cloud10)
greenest20 = coleccion20.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020.add_ee_layer(greenest20, vizParams, 'Mapa sin Nubes')
display(map_2020)

In [51]:
coleccion20_ = coleccion2020.map(Cloud10)
agua20 = coleccion20_.qualityMosaic('NDWI')
map_2020_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020_.add_ee_layer(agua20, vizParams, 'Mapa sin Nubes')
display(map_2020_)

## Mosaico

In [28]:
def llenado(image):
    return image.where(nube).gt(30),greenest21)

In [ ]:
img1 = llenado(agua21)

In [ ]:
map_img1 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img1.add_ee_layer(img1, vizParams, 'Mapa sin Nubes')
display(map_img1)